In [ ]:
from additional.nearest_larger_item import solution_minheap, solution_stack
a = [3,2,1,4]
r1 = solution_minheap(a)
r2 = solution_stack(a)
assert r1 == r2

In [ ]:
a = [1,2,3]
a.pop()

In [ ]:
from additional.robot_up_right import solution_bottom_up, solution_top_down
assert solution_bottom_up(5,4) == solution_top_down(5,4)

In [ ]:
from lesson3.n2_hash_chains import Hash
h = Hash(5)
h('Hell0'), h('world'), h('luck'), h('GooD'), h('Hell0'), h('GooD')

In [ ]:
h.powers

In [ ]:
[ord(c) for c in 'Hell0']

In [ ]:
p = 1000000007
x = 263
powers = [263**i % 1000000007 for i in range(5)]
(72 + (111*powers[1]) + (114 * powers[2]) + (108 * powers[3]) + 100 * powers[4]) % 5

In [ ]:
sum([(ord(ch)*h.powers[i]) for i, ch in enumerate('hell0')]) % 5

In [ ]:
from lesson3.n3_pattern_in_text import solve, Hash

assert solve('aba', 'aba') == [0]
assert solve('abacaba', 'aba') == [0,4]
assert solve('ababacaba', 'aba') == [0,2,6]
assert solve('testTesttesT', 'Test') == [4]
assert solve('baaaaaaa', 'aaaaa') == [1,2,3]

h = Hash(x=500)
s = 'hello'
old_hash = h(s)
ns = 'ellow'
assert h.get_new_hash(len(s), s[0], ns[-1], old_hash) == h(ns)

In [6]:
from lesson4.n4_tree import Tree, Node


def in_order(node: Node):
    if node is None:
        return
    for n in in_order(node.l):
        yield n
    yield node
    for n in in_order(node.r):
        yield n


def show_tree(tree: Tree):
    for n in in_order(tree.root):
        h = max(n.lh, n.rh)
        print(''.join(['\t']*h), repr(n))


def test_tree(t: Tree, result):
    for exp, act in zip(result, in_order(t.root)):
        assert exp == (act.k, act.p.k if act.p else None)


def test_add_tree(nodes, result):
    t = Tree()
    for n in nodes:
        t.add(n)
    test_tree(t, result)


In [2]:
test_add_tree([1], [(1, None)])
test_add_tree([1, 2], [(1, None), (2, 1)])
test_add_tree([2, 1], [(1, 2), (2, None)])
test_add_tree([1, 2, 3], [(1, 2), (2, None), (3, 2)])  # right small rotation
test_add_tree([3, 2, 1], [(1, 2), (2, None), (3, 2)])  # left small rotation
test_add_tree([4, 5, 1, 0, 2, 3], [(0, 1), (1, 2), (2, None),
              (3, 4), (4, 2), (5, 4)])  # left big rotation
test_add_tree([1, 0, 4, 5, 3, 2], [(0, 1), (1, 3), (2, 1),
              (3, None), (4, 3), (5, 4)])  # right big rotation


In [7]:
import random
t = Tree()
nodes = list(range(16))
# random.shuffle(nodes)
print(nodes)
for n in nodes:
    t.add(n)
show_tree(t)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
 0(1)
	 1(3)
 2(1)
		 3(7)
 4(5)
	 5(3)
 6(5)
				 7(null)
 8(9)
	 9(11)
 10(9)
			 11(7)
 12(13)
		 13(11)
	 14(13)
 15(14)
